In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
#Essentials
import numpy as np
import pandas as pd
import pickle
import re
import datetime as dt
from dateutil.relativedelta import *
import time
import seaborn as sns

#SQL related - NEED TO DECIDE WHICH ONE I'LL BE USING AND DELETE THE REST
import sqlite3
import pandas.io.sql as pd_sql
# import psycopg2
# from sqlalchemy import create_engine

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 6000)

import warnings
warnings.filterwarnings('ignore')

In [3]:
def process_duplicates(new_ads_df, existing_df):
    """Takes a dataframe of new advertisments and the dataframe with all the existing ones.
    Returns a new new_ads_df with only truly new records"""
    



def string_to_date(value):
    """Whenever possible, converts text date value to date format (takes "Listen Since" field value as an argument)
    Returns the date value and a boolean True or False to identify whether it successfully converted the date or not."""
    try:
        listed_date = dt.datetime.strptime(value, '%B %d, %Y')
        listed_date = listed_date.date()
        return listed_date, True
    except:
        return np.NaN, False
    


def get_listed_date(value, scraped_date):
    """Converts a string in the 'Listed since' column into a listed_date value (in date format)"""
    today = dt.date.today()

    try:
        listed_date = dt.datetime.strptime(value, '%B %d, %Y').date()
    except:
        if scraped_date is np.NaN:
            listed_date = np.NaN    
        else:         
            if 'Today' in value:
                listed_date = scraped_date
            elif 'week' in value:
                weeks_listed = int(re.search('\d*', value).group())
                listed_date = scraped_date - relativedelta(weeks=weeks_listed)
            elif 'month' in value:
                months_listed = int(re.search('\d*', value).group())
                listed_date = scraped_date - relativedelta(months=months_listed)
            elif '6+' in value:
                # Not precise enough to tell, could be 7 months, could be 2 years
                listed_date = np.NaN
            else:
                listed_date = np.NaN
    return listed_date
    

def get_energy_label(value):
    """Takes 'Energy label' column and strips it from the words 'What does this mean?'
    Return just the label"""
    no_touch_list = ['Not required', 'Not available', np.NaN]
    if value not in no_touch_list:
        return value[0]
    else: 
        return np.NaN

def get_int(value):
    """Trims the price, area and other fields with numbers and converts them into int"""
    try:
        return re.sub('[€\sk.,m²m³v.o.n.permonthBeforeAfter]', '', value)
    except:
        return np.NaN
    

def get_rooms(value, room_type):
    """Retrieves the number of rooms specified by the type (room, bedroom, toilet, bathroom, etc.).
    If bedrooms are not specified and there is only 1 room - returns 0, if there is more than 1 room, but bedrooms not specified - returns NaN.
    Otherwise, returns the number of bedrooms"""
    try:
        value = value.lower()
    except:
        return np.NaN
    
    if room_type == 'room':
        try:
            return int(re.search(f'\d* {room_type}', value).group().strip(f' {room_type}'))
        except:
            return np.NaN
    
    if room_type == 'bedroom':
        try:
            if room_type not in value and int(re.search(f'\d* {room_type}', value).group().strip(f' {room_type}')) == 1:
                return 0
            elif room_type not in value:
                return np.NaN
            else:
                return int(re.search(f'\d* {room_type}', value).group().strip(f' {room_type}'))
        except:
            return np.NaN
    
    if room_type == 'toilet':
        try:
            if room_type not in value and int(re.search(f'\d* {room_type}', value).group().strip(f' {room_type}')) == 1:
                return 1
            elif room_type not in value:
                return np.NaN
            else:
                return int(re.search(f'\d*\s[a-z]*\s?{room_type}', value).group().strip(f' separate {room_type}'))
        except:
            return np.NaN
        
    if room_type == 'bathroom':
        try:
            if room_type not in value and int(re.search(f'\d*\s[a-z]*\s?toilet', value).group().strip(f' separate {room_type}')) == 1:
                return 1
            elif room_type not in value:
                return np.NaN
            else:
                return int(re.search(f'\d* {room_type}', value).group().strip(f' {room_type}'))
        except:
            return np.NaN
        
    
def get_bath_flag(value):
    """Takes Bathroom facilities column and create a Bath_Flag column if a bathtub / bath is available in the property.
    Returns True or False"""
    try:
        if 'bath' in value.lower():
            return True
        else:
            return False
    except:
        return np.NaN
    
    
def get_facilities(value, facility_type):
    """Take the Bathroom facilities column and facility type (toilet, shower, bath, jacuzzi, steam cabin, etc.)
    and returns the number of specified facilities"""
    try:
        value = value.lower()
        facility_type = facility_type.lower()
    except:
        return np.NaN
    
    try:
        return int(re.search(f'\d* {facility_type}', value).group().strip(f' {facility_type}'))
    except:
        if facility_type in value:
            return 1
        else:
            return 0

In [8]:
# First things first, connect to the postgresql database
conn = sqlite3.connect('./Database/ams_market_watch.db')  # You can create a new database by changing the name within the quotes
# cursor = conn.cursor()

# Now pull in existing records from the database table called funda_ads
current_data_df = pd.read_sql_query("""SELECT * FROM funda_ads""", con=conn)

# Also load the data from new_adverts.pkl file 
with open('./Cellar/new_adverts.pkl', 'rb') as new_ads_pkl:
    new_data = pickle.load(new_ads_pkl)

column_list = []

for ad in new_data:
    for feat_name in list(ad.keys()):
        if feat_name not in column_list:
            column_list.append(feat_name)  
    
# Step 3 - Add "Listed_date_present", "listed_date" and "is_duplicate" columns
column_list.append('listed_date')
column_list.append('listed_date_present')
column_list.append('is_duplicate')
column_list.append('is_complete_duplicate')
column_list.append('rank_same_record')


# Step 4 - Initiate the dataframe with the desired columns
new_data_df = pd.DataFrame(columns=column_list)


# Step 5 - Iterate over the ads_list and replace "Listed since" field with a date (using get_listed_date function), where possible 
    #i. #As part of this step, add another key-value pair to identify records with a successful conversion of "Listed since" to date
    #ii. Since we are iterating already, just add the "is_duplicate" column and set it to false
for ad in new_data:
    ad['listed_date'], ad['listed_date_present'] = string_to_date(ad['Listed since'])
    ad['is_duplicate'] = False
    ad['is_complete_duplicate'] = False
    ad['is_price_change'] = False
    ad['is_status_change'] = False
    
    
# Step 6 - iterate over the ads_list and append unique records to the empty dataframe (from step 2) 
    # i. Create a check_ads_list with only columns used to check for duplicates
    # ii. In the process check for duplicates and update "is_duplicate" value, if a record with the same:
        # property_id, title, asking price and status, has already been seen
check_duplicates = ['property_id', 'title']
check_complete_duplicate = ['property_id', 'title', 'Asking price', 'Status']
check_price_change = ['property_id', 'title', 'Status']
check_status_change = ['property_id', 'title', 'Asking price']
ads_seen_list = []

for ad in ads_list:
    # Keeps track of all the ads seen so far (dpl = duplicate)
    ad_check_dpl_value = [ad.get(key) for key in check_duplicates]
    ads_seen_list.append(ad_check_value)
    
    # Assign a rank_same_record value
    ad['rank_same_record'] = ads_seen_list.count(ad_check_value)
    
    # Create a list with columns cords for checking if a record is a complete duplicate (cdpl)
    ad_check_cdpl_value = [ad.get(key) for key in check_complete_duplicate]

    # Creates a smaller list of key values to check if a record already exists, but had a change in price or status (pc = price change, sc = status change)
    ad_check_pc_value = [ad.get(key) for key in check_price_change]
    ad_check_sc_value = [ad.get(key) for key in check_status_change]
    
    # Checks if the ad is in the ads_df already, this updates is_duplicate, is_price_change and is_status_change columns
    if ad_check_dpl_value in ads_df[check_duplicates].values.tolist():
        ad['is_duplicate'] = True
        # Now filter the dataframe to find all the records with the same id and title, and update their "is_duplicate" column value
        ads_df[(ads_df['property_id'] == ad_check_dpl_value[0]) & (ads_df['title'] == ad_check_dpl_value[1])]['is_duplicate'] = True
    if ad_check_cdpl_value in ads_df[check_complete_duplicate].values.tolist():
        ad['is_complete_duplicate'] = True
    if ad['is_complete_duplicate'] == False and ad_check_pc_value in ads_df[['property_id', 'title', 'Status']].values.tolist():
        ad['is_price_change'] = True
    if ad['is_complete_duplicate'] == False and ad_check_sc_value in ads_df[['property_id', 'title', 'Asking price']].values.tolist():
        ad['is_status_change'] = True    
        
    # Finally, append the advert into the dataframe
    ads_df = ads_df.append(ad, ignore_index=True)

,property_link,property_id,title,address,price,neighbourhood,scraped_date,Transfer of ownership,Asking price,Asking price per m²,Listed since,Status,Acceptance,VVE (Owners Association) contribution,Construction,Type apartment,Building type,Year of construction,Type of roof,Surface areas and volume,Areas,Living area,Exterior space attached to the building,Volume in cubic meters,Layout,Number of rooms,Number of bath rooms,Number of stories,Located at,Facilities,Energy,Energy label,Insulation,Heating,Hot water,CH boiler,Cadastral data,Exterior space,Location,Parking,Type of parking facilities,VVE (Owners Association) checklist,Registration with KvK,Annual meeting,Periodic contribution,Reserve fund present,Maintenance plan,Building insurance,Kind of house,Plot size,Bathroom facilities,Garden,Balcony/roof garden,Original asking price,Construction period,Other space inside the building,External storage space,Back garden,Garden location,Storage space,Shed / storage,Facilities_Storage space,Specific,Service charges,Quality marks,Accessibility,Garage,Type of garage,Capacity,Provisional energy label,Sun terrace,Insulation_Storage space,Insulation_Garage,Patio/atrium,Facilities_Garage,Front garden,Rental price,Deposit,Rental agreement
0,https://www.funda.nl/en/koop/amsterdam/apparte...,87214155,Da Costakade 10 C,1052 SJ Amsterdam,"€ 925,000 k.k.","Da Costabuurt, Amsterdam",2020-04-18,,"€ 925,000 k.k.","€ 7,008","April 15, 2020",Available,Available in consultation,€ 150 per month,,Upstairs apartment (apartment),Resale property,1905,Pyramid hip roof covered with slate,,,132 m²,16 m²,378 m³,,4 rooms (3 bedrooms),1 separate toilet,1 story,4th floor,"Mechanical ventilation, TV via cable, air cond...",,C What does this mean?,"Roof insulation, floor insulation and double g...",CH boiler,CH boiler,"HR (gas-fired combination boiler from 2013, in...",AMSTERDAM Q 9040; Ownership situation; Full ow...,,"Alongside water, in center, unobstructed surro...",,Public parking and parking permits,,Yes,Yes,Yes (€ 150 per month),Yes,No,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.funda.nl/en/koop/amsterdam/huis-41...,41738079,Levantkade 86,1019 BG Amsterdam,"€ 1,050,000 k.k.","Oostelijk Havengebied, Amsterdam",2020-04-18,,"€ 1,050,000 k.k.","€ 5,224",2 weeks,Available,Available in consultation,NaN,,NaN,Resale property,1954,NaN,,,201 m²,NaN,686 m³,,8 rooms (4 bedrooms),2 bathrooms and 1 separate toilet,2 stories,NaN,"Jacuzzi, steam cabin, sliding door, skylight a...",,Not available,NaN,"CH boiler, complete floor heating and gas heater",NaN,NaN,PRO PATRIA II B 7630; Area; 291 m²; Ownership ...,,"Alongside water, unobstructed surrounding view...",,Parking permits,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Houseboat, detached residential property",291 m²,"Sauna, 2 showers and toilet",Sun terrace,Roof terrace present,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.funda.nl/en/koop/amsterdam/huis-41...,41642036,Roffart 19,1083 CJ Amsterdam,"€ 800,000 k.k.","Buitenveldert-Oost, Amsterdam",2020-04-18,,"€ 800,000 k.k.","€ 5,333",8 weeks,Available,Available in consultation,NaN,,NaN,Resale property,NaN,Flat roof covered with asphalt roofing,,,150 m²,20 m²,559 m³,,5 rooms (4 bedrooms),2 bathrooms and 1 separate toilet,3 stories,NaN,"Mechanical ventilation, alarm installation, ro...",,C 1.52 What does this mean?,NaN,CH boiler,CH boiler,Intergas Kompakt HRe 36/48 (gas-fired combinat...,AMSTERDAM AK 1242; Area; 162 m²; Ownership sit...,,Alongside a quiet road and in residential dist...,,Paid parking and parking permits,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Desirable residence/villa, row house",162 m²,"Bath, 2 showers and 2 toilets",Back garden and front garden,Balcony present,"€ 850,000 k.k.",1960-1970,15 m²,5 m²,"39 m² (11m deep and 3,5m broad)",Located at the east accessible via the rear,,Attached brick storage,"Heating, electrici

In [ ]:
# Convert columns to the right formats, clean up text values and make them numbers, etc.

clean_ads_df.drop(columns=['price'], inplace=True)
clean_ads_df['property_id'] = clean_ads_df['property_id'].apply(int)
clean_ads_df['listed_date'] = clean_ads_df.apply(lambda x: get_listed_date(x['Listed since'], x['scraped_date']), axis=1)
clean_ads_df['address'] = clean_ads_df['title']+', '+clean_ads_df['address']
clean_ads_df['Asking price'] = clean_ads_df['Asking price'].apply(get_int)
clean_ads_df['Asking price per m²'] = clean_ads_df['Asking price per m²'].apply(get_int)
clean_ads_df['VVE (Owners Association) contribution'] = clean_ads_df['VVE (Owners Association) contribution'].apply(get_int)
clean_ads_df['Year of construction'] = clean_ads_df['Year of construction'].apply(get_int)
clean_ads_df['Living area'] = clean_ads_df['Living area'].apply(get_int)
clean_ads_df['Exterior space attached to the building'] = clean_ads_df['Exterior space attached to the building'].apply(get_int)
clean_ads_df['Volume in cubic meters'] = clean_ads_df['Volume in cubic meters'].apply(get_int)
clean_ads_df['Rooms'] = clean_ads_df['Number of rooms'].apply(get_rooms, room_type='room')
clean_ads_df['Bedrooms'] = clean_ads_df['Number of rooms'].apply(get_rooms, room_type='bedroom')
clean_ads_df['Bathrooms'] = clean_ads_df['Number of bath rooms'].apply(get_rooms, room_type='bathroom')
clean_ads_df['Toilets'] = clean_ads_df['Number of bath rooms'].apply(get_rooms, room_type='toilet')
clean_ads_df['Has_Bathtub'] = clean_ads_df['Bathroom facilities'].apply(get_bath_flag)
clean_ads_df['Baths'] = clean_ads_df['Bathroom facilities'].apply(get_facilities, facility_type='bath')
clean_ads_df['Number of Toilets'] = clean_ads_df['Bathroom facilities'].apply(get_facilities, facility_type='toilet')
clean_ads_df['Showers'] = clean_ads_df['Bathroom facilities'].apply(get_facilities, facility_type='shower')
clean_ads_df['Energy label'] = clean_ads_df['Energy label'].apply(get_energy_label)
clean_ads_df['Provisional energy label'] = clean_ads_df['Provisional energy label'].apply(get_energy_label)


clean_ads_df.rename(columns={'Asking price': 'Asking price (€)', 'Asking price per m²': 'Asking price per m² (€)', 
                       'VVE (Owners Association) contribution': 'VVE contribution (monthly) (€)',
                      'Living area': 'Living area (m²)', 'Volume in cubic meters': 'Volume (m³)'}, inplace=True)